# Classifying Case Outcomes with Flan T5
As 1 of 3 classes
- `'No Relief'`, `'Relief'`, `'Conditional Order'`

In [1]:
!pip3 install transformers datasets
!pip install --upgrade accelerate
!pip uninstall -y transformers accelerate
!pip install transformers accelerate
import transformers
from datasets import load_dataset, load_metric

from google.colab import drive
drive.mount('/content/drive')

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Found existing installation: transformers 4.29.2
Uninstalling transformers-4.29.2:
  Successfully uninstalled transformers-4.29.2
Found existing installation: accelerate 0.19.0
Uninstalling accelerate-0.19.0:
  Successfully uninstalled accelerate-0.19.0
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
  Using cached transformers-4.29.2-py3-none-any.whl (7.1 MB)
  Using cached accelerate-0.19.0-py3-none-any.whl (219 kB)
Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


We'll use Weights and Biases for dislaying logging information during training. You will need to sign up for the service at `wanb.ai`. This will give you an access token which you can paste when running `wanb login` 

In [2]:
!pip3 install wandb
import wandb
wandb.login()

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


wandb: Currently logged in as: kmaurinjones. Use `wandb login --relogin` to force relogin


True

# Creating train/dev/test splits

In [3]:
import pandas as pd

master_data = pd.read_csv("/content/drive/MyDrive/COLAB_FILES/capstone/capstone_data/gold_labels_with_oc_spans.csv")
master_data.head()

,raw_file_name,content,new_case_outcome,outcome_span
0,CEL-62600-16.txt,"[""Arrears Worksheet File Number: CEL-62600-16 ...",No relief,No circumstances were disclosed in accordance ...
1,CEL-62852-16.txt,"[""Arrears Worksheet File Number: CEL-62852-16 ...",Relief,I have considered all of the disclosed circums...
2,CEL-63024-16.txt,"[""Arrears Worksheet File Number: CEL-63024-16 ...",Relief,I have considered all of the disclosed circums...
3,CEL-63056-16.txt,"[""Arrears Worksheet File Number: CEL-63056-16 ...",No relief,I have considered all of the disclosed circums...
4,CEL-63193-16.txt,"[""Arrears Worksheet File Number: CEL-63193-16 ...",No relief,I have considered all of the disclosed circums...


In [4]:
master_data['outcome_span'][0]

'No circumstances were disclosed in accordance with subsection 83(2) of the Act to consider relief from eviction pursuant to subsection 83(1) of the Act.'

In [5]:
train_split = 0.75
dev_split = 0.10
test_split = 0.15

train_rows = int(len(master_data) * train_split)
dev_rows = int(len(master_data) * dev_split)
test_rows = int(len(master_data) * test_split)

train_df = master_data.iloc[:train_rows, :]
dev_df = master_data.iloc[train_rows:train_rows + dev_rows, :]
test_df = master_data.iloc[train_rows + dev_rows:, :]

file_dir = "/content/drive/MyDrive/COLAB_FILES/capstone/capstone_data/"

train_df.to_csv(file_dir + "gold_labels_with_oc_spans_train.csv", index = False)
dev_df.to_csv(file_dir + "gold_labels_with_oc_spans_dev.csv", index = False)
test_df.to_csv(file_dir + "gold_labels_with_oc_spans_test.csv", index = False)

# making sure all rows are accounted for across the 3 sets
assert len(train_df) + len(dev_df) + len(test_df) == len(master_data)

In [6]:
test_df.head()

,raw_file_name,content,new_case_outcome,outcome_span
571,TSL-07120-19.txt,['Order under Section 69 Residential Tenancies...,No relief,"Accordingly, I have considered all of the disc..."
572,TSL-07388-19.txt,"[""Order under Section 21.2 of the Statutory Po...",No relief,The Tenant alleged that the Member made the fo...
573,TSL-07620-19.txt,"[""Order under Section 69 Residential Tenancies...",No relief,I have considered all of the disclosed circums...
574,TSL-08283-19-RV.txt,"[""Order under Section 21.2 of the Statutory Po...",Relief,The review was granted on this basis.
575,TSL-08760-19.txt,"[""Order under Section 69 Residential Tenancies...",No relief,I have considered all of the disclosed circums...


In [7]:
from datasets import load_dataset
# train_data = load_dataset(path = ".", data_files = train_df)['train'] # "train" is an artifact of the dataloader
# dev_data = load_dataset(path = ".", data_files = dev_df)['train']
# test_data = load_dataset(path = ".", data_files = test_df)['train']
train_data = load_dataset(path = ".", data_files = "/content/drive/MyDrive/COLAB_FILES/capstone/capstone_data/gold_labels_with_oc_spans_train.csv")['train'] # "train" is an artifact of the dataloader
dev_data = load_dataset(path = ".", data_files = "/content/drive/MyDrive/COLAB_FILES/capstone/capstone_data/gold_labels_with_oc_spans_dev.csv")['train']
test_data = load_dataset(path = ".", data_files = "/content/drive/MyDrive/COLAB_FILES/capstone/capstone_data/gold_labels_with_oc_spans_test.csv")['train']

Extracting data files:   0%|          | 0/1 [00:00<?, ?it/s]

Generating train split: 0 examples [00:00, ? examples/s]

Dataset csv downloaded and prepared to /root/.cache/huggingface/datasets/csv/.-201430a00468b563/0.0.0/6954658bab30a358235fa864b05cf819af0e179325c740e4bc853bcc7ec513e1. Subsequent calls will reuse this data.


  0%|          | 0/1 [00:00<?, ?it/s]

Extracting data files:   0%|          | 0/1 [00:00<?, ?it/s]

...

  0%|          | 0/1 [00:00<?, ?it/s]

Extracting data files:   0%|          | 0/1 [00:00<?, ?it/s]

Generating train split: 0 examples [00:00, ? examples/s]

Dataset csv downloaded and prepared to /root/.cache/huggingface/datasets/csv/.-e72dbeeb1122093b/0.0.0/6954658bab30a358235fa864b05cf819af0e179325c740e4bc853bcc7ec513e1. Subsequent calls will reuse this data.


  0%|          | 0/1 [00:00<?, ?it/s]

Checking an example to make sure it's in the right format:

In [8]:
print(test_data[0])

{'raw_file_name': 'TSL-07120-19.txt', 'content': '[\'Order under Section 69 Residential Tenancies Act, 2006 File Number: TSL-07120-19 L1 Application M.B.M G of C Ltd.\', "(the \'Landlord\') applied for an order to terminate the tenancy and evict J (J) M and K M (the \'Tenants\') because the Tenants did not pay the rent that the Tenants owe.", \'The Landlord also claimed NSF cheque charges and related administration charges.\', \'L2 Application The Landlord also applied for an order to terminate the tenancy and evict the Tenants because they, another occupant of the rental unit or someone they permitted in the residential complex have: • substantially interfered with the reasonable enjoyment or lawful right, privilege or interest of the Landlord or another tenant; • wilfully or negligently caused undue damage to the premises; and • seriously impaired the safety of any person.\', \'The Landlord also applied for an order to terminate the tenancy and evict the Tenants because they have bee

We'll now start to process our data. We will need to use `nltk.word_tokenize`, so we need to load the `nltk` module. Additionally, we'll load the Flan T5 tokenizer.

In [9]:
import nltk
nltk.download('punkt')
import string
from transformers import AutoTokenizer

# model_checkpoint = "google/flan-t5-base"
model_checkpoint = "google/flan-t5-small" # trying small model to see if performance is as good as base size -- might be a better tradeoff
tokenizer = AutoTokenizer.from_pretrained(model_checkpoint)

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [10]:
# PREFIX = "extract metadata boundary:"
PREFIX = "classify case outcome:"
MAX_INPUT_LENGTH = 512 # max of the model
MAX_TARGET_LENGTH = 512 # max of the model

def preprocess_data(examples):
    inputs = [PREFIX + " " + text for text in examples["outcome_span"]] # prefix + extracted case outcome span
    
    model_inputs = tokenizer(inputs, 
                             max_length = MAX_INPUT_LENGTH, 
                             truncation = True) # truncating input (outcome span) if necessary -- shouldn't be necessary but just in case

    # Setup the tokenizer for targets
    with tokenizer.as_target_tokenizer():
        labels = tokenizer(examples["new_case_outcome"], # clean metadata as target "label"
                           max_length = MAX_TARGET_LENGTH, 
                           truncation = True) # shouldn't make a difference -- the outcome should always be one of the 3, which is only a few tokens at most

    model_inputs["labels"] = labels["input_ids"]
    
    return model_inputs

We'll then apply the tokenization function to our datasets

In [11]:
tokenized_train = train_data.map(preprocess_data, batched = True)
tokenized_dev = dev_data.map(preprocess_data, batched = True)

Map:   0%|          | 0/504 [00:00<?, ? examples/s]

/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:3606: UserWarning: `as_target_tokenizer` is deprecated and will be removed in v5 of Transformers. You can tokenize your labels by using the argument `text_target` of the regular `__call__` method (either in the same call as your input texts if you use the same keyword arguments, or in a separate call.
  warnings.warn(


Map:   0%|          | 0/67 [00:00<?, ? examples/s]

Let's print the first tokenized training example:

In [12]:
tokenized_train[0]

{'raw_file_name': 'CEL-62600-16.txt',
 'content': '["Arrears Worksheet File Number: CEL-62600-16 Time period for Arrears Owing From: September 1, 2016 to November 5, 2016 (From the commencement of arrears to the termination date in the notice, or the end of the rental period if the tenancy is not being terminated.) Part 1 - Calculations of Arrears Owing (A) Rent Period (monthly, weekly, etc.) (B) Rent Charged (C) Lawful Rent (if issue raised) (D) Lower of (B) and (C) (E) Rent Paid (F) Amount Owing (D-E) 01/09/2016 - 30/09/2016 $1,760.00 $1,760.00 $1,760.00 01/10/2016 - 31/10/2016 $1,760.00 $1,760.00 $1,760.00 01/11/2016 - 05/11/2016 $289.32 $289.32 $289.32 **Part Month ** To calculate the Rent for part of a month, use the following formula for columns (B), (C) and (D): Monthly Rent X 12 X # Days 365 (F) Total Rent Owing $3,809.32 (G1) Arrears Owing $3,809.32 [From (F)] (G2) Arrears Claimed $5,450.00 (G3) Include whichever is less when Calculation Total Arrears Owing $3,809.32 (H1) Paym

Next we will import classes which are used for fine-tuning of seq2seq models like Flan-T5.

* `AutoModelForSeq2SeqLM` loads the model itself,
* `DataCollatorForSeq2Seq` takes care of data batching,
* `Seq2SeqTrainingArguments` sets all hyperparameters for training, and
* `Seq2SeqTrainer` trains the model

In [13]:
from transformers import AutoModelForSeq2SeqLM, DataCollatorForSeq2Seq, Seq2SeqTrainingArguments, Seq2SeqTrainer

We'll then set the hyperparameters for model training:

```
model_dir -- where to save model checkpoints
evaluation_strategy="steps" -- evaluate every N steps
eval_steps=100 -- where N = 100
logging_strategy="steps" -- write information about training loss  every N steps
logging_steps=1 -- where N = 1
save_strategy="steps" -- save model every N steps
save_steps=100 -- where N = 100
learning_rate=4e-5 -- initial learning rate for training
per_device_train_batch_size=batch_size -- training batch size
per_device_eval_batch_size=batch_size -- evaluation batch size
weight_decay=0.01 -- hyperparameter for weight decay during training
save_total_limit=3 -- save a maximum of 3 models
num_train_epochs=1 -- number of training epochs
predict_with_generate=True -- generate the actual output during evaluation 
```

In [14]:
# !pip install --upgrade accelerate
# !pip uninstall -y transformers accelerate
# !pip install transformers accelerate
from transformers import AutoModelForSeq2SeqLM, DataCollatorForSeq2Seq, Seq2SeqTrainingArguments, Seq2SeqTrainer

In [15]:
batch_size = 8
model_name = "flan-t5-outcome_classifier_small"
model_dir = f"/content/drive/MyDrive/COLAB_FILES/capstone/capstone_data/outcome_classification_models/{model_name}"

args = Seq2SeqTrainingArguments(
    model_dir,
    evaluation_strategy = "steps",
    eval_steps = 150,
    logging_strategy = "steps",
    logging_steps = 1,
    save_strategy = "steps",
    save_steps = 150,
    learning_rate = 4e-5,
    per_device_train_batch_size = batch_size,
    per_device_eval_batch_size = batch_size,
    weight_decay = 0.01,
    save_total_limit = 3,
    num_train_epochs = 10,
    predict_with_generate = True)

Let's define a data collator which batches the training and development data for us:

In [16]:
data_collator = DataCollatorForSeq2Seq(tokenizer, padding = True)

In [17]:
# This is potentially required because of some obscure colab bug
import locale
locale.getpreferredencoding = lambda: "UTF-8"

# !pip3 install sacrebleu
# metric = load_metric("sacrebleu") # using f1 instead of bleu for this task

Using F1 Macro for this task because of the class imbalance.

F1 Macro: This metric calculates the F1 score for each class individually and then takes the average across all classes. It treats each class equally and does not consider the class distribution. F1 macro is suitable when you want to give equal importance to each class, regardless of its size. It provides a balanced evaluation by considering all classes, but it might not reflect the performance accurately if you are primarily interested in the minority class (1%) and its correct classification.

In [18]:
# import scoring-related things

from sklearn.metrics import f1_score
from sklearn.metrics import classification_report

In [19]:
import numpy as np

def compute_metrics(eval_pred):
    predictions, labels = eval_pred
    decoded_preds = tokenizer.batch_decode(predictions, skip_special_tokens = True)
    
    # The labels are padded to equal length using a special symbol -100.
    # This is not the regular <PAD> symbol for the tokenizer so
    # we'll have to replace -100 in the labels before decoding.
    labels = np.where(labels != -100, labels, tokenizer.pad_token_id)
    decoded_labels = tokenizer.batch_decode(labels, skip_special_tokens = True)

    # For BLEU, we'll need to split the outputs into sequences
    # of word tokens. We'll just use NLTK word_tokenize.
    decoded_preds = [nltk.word_tokenize(pred.strip())
                      for pred in decoded_preds]
    decoded_labels = [[nltk.word_tokenize(label.strip())]
                      for label in decoded_labels]
    
    # Compute BLEU scores
    # result = metric.compute(predictions = decoded_preds, references = decoded_labels)

    # Compute f1 macro as eval score
    result = f1_score(y_true = decoded_labels, y_pred = decoded_preds, average = "macro")

    # Return BLEU scores
    return result 

We'll then initalize a trainer for our model. It takes a function `model_init` which returns a plain non-fine-tuned Flan T5 model

In [20]:
# Function that returns an untrained model to be trained
def model_init():
    return AutoModelForSeq2SeqLM.from_pretrained(model_checkpoint)

trainer = Seq2SeqTrainer(
    model_init = model_init,
    args = args,
    train_dataset = tokenized_train,
    eval_dataset = tokenized_dev,
    data_collator = data_collator,
    tokenizer = tokenizer,
    compute_metrics = compute_metrics
)

In [21]:
PREFIX

'classify case outcome:'

Testing how the model does before any finetuning -- it shouldn't be familiar with these labels

In [22]:
model = model_init()
text = train_data[0]['outcome_span']
inputs = [PREFIX + text] # PREFIX = "classify case outcome:"

print("INPUT:", inputs)
inputs = tokenizer(inputs, max_length = 512, truncation = True, return_tensors = "pt")
output = model.generate(**inputs, num_beams = 8, do_sample = True, min_length = 1, max_length = 64)
decoded_output = tokenizer.batch_decode(output, skip_special_tokens = True)[0]
print("OUTPUT:", decoded_output)

INPUT: ['classify case outcome:No circumstances were disclosed in accordance with subsection 83(2) of the Act to consider relief from eviction pursuant to subsection 83(1) of the Act.']
OUTPUT: case outcome: No circumstances were disclosed in accordance with subsection 83(2) of the Act to consider relief from eviction pursuant to subsection 83(1) of the Act.


Training -- use wandb link to track training progress

In [ ]:
trainer.train()

/usr/local/lib/python3.10/dist-packages/transformers/optimization.py:407: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(


After training, we can load the best checkpoint model from Google Drive:

In [ ]:
print(model_dir)

In [ ]:
# using the same model_dir as before

tokenizer = AutoTokenizer.from_pretrained(model_dir)
model = AutoModelForSeq2SeqLM.from_pretrained(model_dir)

# max_input_length = 128

Let's see how it performs on a given test example

In [ ]:
test_case = test_data[110]

text = test_case['outcome_spann']
print(f"GOAL: {test_case['new_case_outcome']}")
inputs = [PREFIX + text] # PREFIX = "extract metadata boundary:"

print("INPUT:", inputs)
inputs = tokenizer(inputs, max_length = 512, truncation = True, return_tensors = "pt")
output = model.generate(**inputs, num_beams = 8, do_sample = True, min_length = 1, max_length = 128)
decoded_output = tokenizer.batch_decode(output, skip_special_tokens = True)[0]
print("OUTPUT:", decoded_output)